In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
df = spark.read.json("people.json")

In [4]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.columns

['age', 'name']

In [7]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## Define our own schema format to import information

In [9]:
# If we need to clarify the Schema of our DataFrame
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [10]:
# Define the Schema that I am expecting
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]

In [11]:
final_struct = StructType(fields=data_schema)

In [12]:
# Now we will read the json file but with the expected schema
df = spark.read.json('people.json',schema=final_struct)

In [13]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [14]:
# Get a column
df['age']

Column<b'age'>

In [15]:
# We get the column object
type(df['age'])

pyspark.sql.column.Column

In [16]:
# Use the select object
df.select('age')

DataFrame[age: int]

In [17]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [18]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [20]:
# The first 2 rows of the DataFrame
df.head(2)[0]

Row(age=None, name='Michael')

In [21]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [22]:
# Select multiple columns
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



If I want to print the element of a DataFrame I should use the method show, if I am getting the rows of a DataFrame I should use the indenteition number.

printing from a DataFrame `df.select('age').show()`

printing from a row `df.head(2)[0]`

### Adding a column

In [24]:
# withColumn returns a new DataFrame by adding a new column or replacing an existing column.
# The first thing you pass in is the name of your new column
# then you pass in a column
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



Everytime we do those operations we are creating a copy of the DataFrame, the operations are not in place, we should save those changes into a new variable.

In [26]:
# To rename a column
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [27]:
df.withColumn('new_stuff', df['age']+2).show()

+----+-------+---------+
| age|   name|new_stuff|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       32|
|  19| Justin|       21|
+----+-------+---------+



## Using SQL to extract information from DataFrame
### Spark SQL

In [28]:
# In order to use SQL to extract information
df.createOrReplaceTempView('people')

In [29]:
results = spark.sql('SELECT * FROM people')

In [30]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [31]:
new_results = spark.sql('SELECT * FROM people WHERE age=30')

In [32]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

